# BrewER: Entity Resolution On-Demand

In [1]:
import brewer

In [2]:
import ipywidgets as widgets
import pandas as pd
import re
import sys
import variables as var

from IPython.display import clear_output, display, HTML
from pprint import pprint

pd.set_option("display.max_rows", None)


def top_k_parser(input_query):
    query_string = input_query.value.lower()
    query_tokens = [token for token in re.split("\s|;|,|\n", query_string) if token != ""]
    
    top_k = -1
    
    alert = False
    for token in query_tokens:
        if alert:
            top_k = token
            break
        if token == "top":
            alert = True
    
    try:
        top_k = int(top_k)
        if top_k <= 0:
            top_k = -1
    except ValueError:
        top_k = -1
    
    return top_k


def parser(input_query):
    
    query = dict()
    query["complete"] = False
    
    query_string = input_query.value.lower()
    query_tokens = [token for token in re.split("\s|;|,|\n", query_string) if token != ""]

    brewer_query = True if "group by entity" in query_string else False
    
    if len(query_tokens) == 0:
        return query
    
    i = 0
    
    if query_tokens[i] != "select":
        return query
    
    if i < len(query_tokens) - 1:
        i += 1
    else:
        return query
    
    if query_tokens[i] == "top":
        if i < len(query_tokens) - 1:
            i += 1
        else:
            return query
        try:
            query["top_k"] = int(query_tokens[i])
            if query["top_k"] <= 0:
                return query
            if i < len(query_tokens) - 1:
                i += 1
            else:
                return query
        except ValueError:
            return query
    else:
        query["top_k"] = -1
    
    if query_tokens[i] == "*":
        if brewer_query:
            return query
        else:
            attributes = ["*"]
            if i < len(query_tokens) - 1:
                i += 1
            else:
                return query
    else:
        attributes = []
        while query_tokens[i] != "from":
            attributes.append(query_tokens[i])
            if i < len(query_tokens) - 1:
                i += 1
            else:
                return query
    
    if query_tokens[i] != "from":
        return query
    else:
        if i < len(query_tokens) - 1:
            i += 1
        else:
            return query
        
    if query_tokens[i] in var.datasets.keys():
        query["ds"] = query_tokens[i]
        if i < len(query_tokens) - 1:
            i += 1
        else:
            query["complete"] = True
    else:
        return query
    
    query["ordering_key"] = var.datasets[query["ds"]]["default_ordering_key"]
    query["ordering_mode"] = var.datasets[query["ds"]]["default_ordering_mode"]
    
    if attributes == ["*"]:
        query["attributes"] = var.datasets[query["ds"]]["attributes"]
    else:
        query["attributes"] = []
        if not brewer_query:
            for attribute in attributes:
                if attribute not in var.datasets[query["ds"]]["attributes"] or attribute in query["attributes"]:
                    query["complete"] = False
                    return query
                else:
                    query["attributes"].append(attribute)
        else:
            query["aggregation_functions"] = dict()
            for attribute in attributes:
                attribute_tokens = attribute.split("(", 1)
                if len(attribute_tokens) != 2:
                    query["complete"] = False
                    return query
                if attribute_tokens[1][:-1] not in var.datasets[query["ds"]]["attributes"] or attribute_tokens[1][:-1] in query["attributes"]:
                    query["complete"] = False
                    return query
                if attribute_tokens[0].upper() not in var.aggregation_functions:
                    query["complete"] = False
                    return query
                else:
                    query["attributes"].append(attribute_tokens[1][:-1])
                    query["aggregation_functions"][attribute_tokens[1][:-1]] = attribute_tokens[0]
    
    if query["complete"]:
        return query
    
    if brewer_query:
        if i + 5 < len(query_tokens):
            if [query_tokens[j] for j in range(i, i + 5)] != ["group", "by", "entity", "with", "matcher"]:
                query["complete"] = False
                return query
            if query_tokens[i + 5] in var.pipelines.keys():
                query["pipeline"] = query_tokens[i + 5]
                query["blocking_function"] = var.pipelines[query["pipeline"]]["blocking_function"]
                query["matching_function"] = var.pipelines[query["pipeline"]]["matching_function"]
            else:
                query["complete"] = False
                return query
            if i + 6 < len(query_tokens) - 1:
                i += 6
            else:
                query["complete"] = True
                return query
        else:
            query["complete"] = False
            return query
    else:
        query["aggregation_functions"] = dict()
        query["blocking_function"] = "None (Cartesian Product)"
        query["matching_function"] = "None (Dirty)"
    
    if "order" in query_tokens[i:]:
        index_order = query_tokens[i:].index("order")
        where_clause = query_tokens[i : i + index_order]
        order_by_clause = query_tokens[i + index_order:]
    else:
        where_clause = query_tokens[i:]
        order_by_clause = list()
    
    if len(order_by_clause) > 0 and len(order_by_clause) != 4:
        query["complete"] = False
        return query
    else:
        if order_by_clause[0:2] != ["order", "by"]:
            query["complete"] = False
            return query
        if order_by_clause[3] not in ["asc", "desc"]:
            query["complete"] = False
            return query
        else:
            query["ordering_mode"] = order_by_clause[3]
        if not brewer_query:
            if order_by_clause[2] in var.datasets[query["ds"]]["attributes"]:
                query["ordering_key"] = order_by_clause[2]
            else:
                query["complete"] = False
                return query
        else:
            attribute_tokens = order_by_clause[2].split("(", 1)
            if len(attribute_tokens) != 2:
                query["complete"] = False
                return query
            if attribute_tokens[0].upper() not in var.aggregation_functions:
                query["complete"] = False
                return query
            if attribute_tokens[1][:-1] not in var.datasets[query["ds"]]["attributes"]:
                query["complete"] = False
                return query
            if attribute_tokens[1][:-1] in query["aggregation_functions"].keys():
                if query["aggregation_functions"][attribute_tokens[1][:-1]] != attribute_tokens[0]:
                    query["complete"] = False
                    return query
            else:
                query["aggregation_functions"][attribute_tokens[1][:-1]] = attribute_tokens[0]
            query["ordering_key"] = attribute_tokens[1][:-1]

    # # Parse the WHERE clause
    # query["conditions"] = list()
    # where_clause = list()
    # if i + 1 < len(query_string_tokens):
    #     i += 1
    #     if query_string_tokens[i] == "where":
    #         i += 1
    #         while i < len(query_string_tokens) and query_string_tokens[i] != "order":
    #             where_clause.append(query_string_tokens[i])
    #             i += 1
    # if len(where_clause) > 0:
    #     j = 0
    #     while j < len(where_clause):
    #         if where_clause[j] not in var.datasets[ds]["attributes"] or where_clause[j + 1] not in ["like", "=", "!=", ">", ">=", "<", "<="]:
    #             raise_error()
    #         if j + 3 < len(where_clause):
    #             if where_clause[j + 3] not in ["and", "or"]:
    #                 raise_error()
    #         j += 4
    #     j = 0
    #     while j < len(where_clause):
    #         query["conditions"].append((where_clause[j], where_clause[j + 2], where_clause[j + 1]))
    #         j += 4
    #     if len(where_clause) > 3:
    #         query["operator"] = where_clause[3]
    #     else:
    #         query["operator"] = "or"

    if len(where_clause) > 0:
        j = 0
        if brewer_query and where_clause[0] != "having":
            query["complete"] = False
            return query
        elif not brewer_query and where_clause[0] != "where":
            query["complete"] = False
            return query
        
        where_clause = where_clause[1:]
        j = 0
        
        # if j < len(where_clause) - 1:
        #         j += 1
        # else:
        #     query["complete"] = False
        #     return query
        
        conditions = list()
        
        while j < len(where_clause):
            if where_clause[j + 1] not in ["like", "=", "!=", ">", ">=", "<", "<="]:
                query["complete"] = False
                return query
            if j + 3 < len(where_clause):
                if where_clause[j + 3] not in ["and", "or"]:
                    query["complete"] = False
                    return query
            j += 4
        j = 0
        while j < len(where_clause):
            conditions.append((where_clause[j], where_clause[j + 2], where_clause[j + 1]))
            j += 4
        if len(where_clause) > 3:
            query["operator"] = where_clause[3]
        else:
            query["operator"] = "or"
        
        query["conditions"] = list()
        for condition in conditions:
            if not brewer_query:
                if condition[0] not in var.datasets[query["ds"]]["attributes"]:
                    query["complete"] = False
                    return query
                else:
                    query["conditions"].append(condition)
            else:
                attribute_tokens = condition[0].split("(", 1)
                if len(attribute_tokens) != 2:
                    query["complete"] = False
                    return query
                if attribute_tokens[0].upper() not in var.aggregation_functions:
                    query["complete"] = False
                    return query
                if attribute_tokens[1][:-1] not in var.datasets[query["ds"]]["attributes"]:
                    query["complete"] = False
                    return query
                if attribute_tokens[1][:-1] in query["aggregation_functions"].keys():
                    if query["aggregation_functions"][attribute_tokens[1][:-1]] != attribute_tokens[0]:
                        query["complete"] = False
                        return query
                query["conditions"].append((attribute_tokens[1][:-1], condition[1], condition[2]))

    query["complete"] = True
    return query

input_query = widgets.Textarea(
    placeholder="Write your query here",
    # description="Query:",
    disabled=False,
    layout=widgets.Layout(width="99%", height="125px")
)

run_brewer = widgets.Button(
    description="Run",
    disabled=False,
    button_style="",
    # tooltip="Click me",
    style=dict(button_color = "springgreen", font_weight="bold")
)

resume_brewer = widgets.Button(
    description="Resume",
    disabled=False,
    button_style="",
    # tooltip="Click me",
    style=dict(button_color = "red", font_weight="bold")
)

clear_brewer = widgets.Button(
    description="Clear",
    disabled=False,
    button_style="",
    # tooltip="Click me",
    style=dict(button_color = "lightskyblue", font_weight="bold")
)

brewer_output = widgets.Output()

def on_run_brewer_clicked(b):
    with brewer_output:
        brewer.run(parser(input_query), mode="scratch", top_k=top_k_parser(input_query))

run_brewer.on_click(on_run_brewer_clicked)

def on_resume_brewer_clicked(b):
    with brewer_output:
        brewer.run(parser(input_query), mode="resume", top_k=top_k_parser(input_query))

resume_brewer.on_click(on_resume_brewer_clicked)

def on_clear_brewer_clicked(b):
    with brewer_output:
        clear_output()

clear_brewer.on_click(on_clear_brewer_clicked)

brewer_buttons = widgets.HBox(
    [run_brewer, resume_brewer, clear_brewer],
    layout=widgets.Layout(
        display="flex",
        flex_flow="row",
        align_items="center",
        justify_content="center",
        width="99%"
    )
)

display(input_query)
display(brewer_buttons)
display(brewer_output)

Textarea(value='', layout=Layout(height='125px', width='99%'), placeholder='Write your query here')

Output()